In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

C:\Users\shanm\AppData\Local\Temp\ipykernel_2272\2129287977.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
reviews_sample= pd.read_csv(r"C:\Users\shanm\OneDrive\Desktop\Hemanth\Git_data\Sephora_sample\X_reviews_sample.csv",low_memory =True)
reviews_sample=reviews_sample.head(1000)

In [3]:
from transformers import BertTokenizerFast, TFBertModel,BatchEncoding,TFBertEmbeddings,BertConfig

In [4]:
tokenize= BertTokenizerFast.from_pretrained('bert-base-uncased')

In [5]:
encoded_dict=tokenize([x for x in reviews_sample.clean_text],max_length=60,padding='max_length',truncation=True,return_tensors='tf')

In [6]:
tokenize.convert_ids_to_tokens(tokenize('Hi! wlecome to the world')['input_ids'])

['[CLS]',
 'hi',
 '!',
 'w',
 '##le',
 '##com',
 '##e',
 'to',
 'the',
 'world',
 '[SEP]']

In [7]:
np.array(encoded_dict['input_ids']).shape

(1000, 60)

In [8]:
len(encoded_dict['input_ids'][0]),len(encoded_dict['input_ids'][1])

(60, 60)

In [9]:
model=TFBertModel.from_pretrained('bert-base-uncased')
output_embedding=model(encoded_dict['input_ids'],encoded_dict['attention_mask'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # encoding
y_label= le.fit_transform(reviews_sample.label)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dropout,Dense
from tensorflow.keras.optimizers import Adam
model = Sequential()
# model.add(Embedding(max_features,64,input_length = 200))
# model.add(Bidirectional(LSTM(12, return_sequences = True), input_shape=(200, 64)))
# model.add(Bidirectional(LSTM(20),input_shape=(200, 12)))
model.add(Dense(768, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(np.round(768/4), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(np.round(768/8), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])


In [12]:
model.fit(output_embedding.pooler_output,y_label, epochs = 5)

Epoch 1/5


32/32 [==============================] - 2s 12ms/step - loss: 0.7350 - accuracy: 0.5020
Epoch 2/5
32/32 [==============================] - 0s 11ms/step - loss: 0.7136 - accuracy: 0.5240
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.7034 - accuracy: 0.5160
Epoch 4/5
32/32 [==============================] - 0s 13ms/step - loss: 0.6768 - accuracy: 0.5570
Epoch 5/5
32/32 [==============================] - 0s 12ms/step - loss: 0.6846 - accuracy: 0.5630


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               590592    
                                                                 
 dropout_37 (Dropout)        (None, 768)               0         
                                                                 
 dense_1 (Dense)             (None, 192)               147648    
                                                                 
 dropout_38 (Dropout)        (None, 192)               0         
                                                                 
 dense_2 (Dense)             (None, 96)                18528     
                                                                 
 dropout_39 (Dropout)        (None, 96)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 9